# Tutorial on basic topics

## Importing the library
At the beginning import the library (it is assumed you have already installed it - see https://neuroinflab.wordpress.com/research/pymice/ for details). The name _`pm`_ will be assigned to the library.

In [ ]:
import pymice as pm

## Downloading example IntelliCage data.
Before you can start the tutorial you have to have the training data stored in your working directory.

The library might do it for you after you call _`getTutorialData()`_ function.

In [ ]:
pm.getTutorialData(quiet=True)

## Exploring data from an IntelliCage archive.
At the beginning load data from one of files to investigate it.

In [ ]:
ml = pm.Loader('C57_AB/2012-08-28 15.33.58.zip')

### Visits
#### Obtaining visit objects
The most important piece of data obtained from IntelliCage system are recordings of visits performed by mice to corners. You can obtain list of objects representing all recorded visits with _`.getVisits()`_ method.

**WARNING: The visits might be in a random order!**

For details on selecting a subset of available visits and on ordering the visits see _"Tutorial on advanced topics"_

In [ ]:
visits = ml.getVisits()
print(type(visits))
print(len(visits))
visit = visits[0]
print(type(visit))
print(repr(visit))

#### Visit objects
Every _`Visit`_ object contains information about one visit to a corner.

The _`.Start`_ and _`.End`_ attributes of the visit object are respectively start and end times of the visit and they are instances of _`datatime.datatime`_ class. The _`.Duration`_ attribute is their derivative (and therefore a _`datatime.timedelta`_ object).

The _`.Cage`_ and _`.Corner`_ attributes indicates whhich corner of which cage was visited; they are guaranted to be convertable to _`int`_.

Another important attribute is _`.Animal`_ which is an _`Animal`_ object representing the mouse performing the visit.

In [ ]:
print("visit of %s in cage #%d to corner #%d" % (visit.Animal, visit.Cage, visit.Corner))
print("visit duration: %.2fs" % visit.Duration.total_seconds())
print("(from %s to %s)" % (visit.Start, visit.End))

### Animals
#### Animal objects
An _`Animal`_ object represents a mouse housed in the system, which contains basic information about the animal.

Names of its _`.Name`_ and _`Sex`_ attributes are self-explanatory (both are instances of _`unicode`_ class). The _`Tag`_ attribute is a set of animal's transponder identificators (containing more than one identificator if mouse's transponder has been changed during the experiment).

In [ ]:
animal = visit.Animal
print(type(animal))
print(repr(animal))
print(animal.Name)
print(animal.Sex)
print(animal.Tag)

#### Registered animals
You can directly access an _`Animal`_ object associated with any mouse registered in the system with the _`.getAnimal()`_ method. You can also use the method to obtain names of all animals registered in the system.

In [ ]:
animal = ml.getAnimal('C57 A 1')
print(repr(animal))
print(animal.Name)
print(animal.Sex)
print()

print('All registered mice:')
for name in ml.getAnimal():
    print(name)

#### Text representation of Animal objects
It is also possible to use "shortcuts" to access the information.

In [ ]:
print(str(animal))
print(str(animal) == animal.Name)

### Cages
#### Animals and cages
Animals are housed in cages and with _`.getCage()`_ method you can check in which cage(s) the animal was detected.
The cage object is either guaranted to be convertable to an integer or to be a collection of such objects.  
You can also check (with _`.getInmates()`_ method) which mice were housed in the cage.

In [ ]:
cage = ml.getCage('C57 A 1')
print(int(cage))
print()
print("Mice housed in cage #%d:" % cage)
mice = ml.getInmates(cage)
for mouse in mice:
    print(mouse)

#### Available cages
The method (_`.getInmates()`_) can be also used to list available cages.

In [ ]:
print(ml.getInmates())

### Animal groups
Animals might be also assigned to certain groups. To list them (as well to obtaing object containing information about a particular group), use the _`.getGroup()`_ method.


In [ ]:
print(ml.getGroup())
group = ml.getGroup('C57 A')
print("Animals of group %s:" % group.Name)
for mouse in group.Animals:
    print(repr(mouse))

## Exploring nosepoke data
If the noseopke data are loaded (see _"Tutorial on advanced topics"_ for details), a _`Visit`_ object has a _`.Nosepoke`_ attribute containing tuple of objects representing nosepoke events.

**WARNING: The nosepokes might be in a random order.**

The _`order`_ parameter of _`.getVisits()`_ method is there **solely for technical purposes of the tutorial** - it enforces the order of visits so the fifth one (of index 4) is always the same.

In [ ]:
visits = ml.getVisits(order=('Start', 'End'))
visit = visits[4]
print(type(visit.Nosepokes))
print(len(visit.Nosepokes))

_`.Start`_, _`.End`_ and _`.Duration`_ attributes of a _`Nosepoke`_ object are analogous to those of the visit object. _`.Side`_ attribute is guaranted to be convertable to _`int`_. The _`.Visit`_ attribute is the same visit object the nosepoke is assigned to.

_`.Door`_ attribute is an auxilary attribute indicating which (left or right) side of the corner was nosepoked.
The _`Visit`_ object also provides several auxilary aggregate attributes providing summary of corresponding attributes of its _`Nosepoke`_ objects.

In [ ]:
visit = visits[4]
nosepoke = visit.Nosepokes[1]
print(type(nosepoke))
print(repr(nosepoke))

from datetime import timedelta
print("nosepoke to side #%d (%s) of the cage #%d" % (nosepoke.Side, nosepoke.Door, nosepoke.Visit.Cage))
print("nosepoke duration: %s (from %s to %s)" % (nosepoke.Duration, nosepoke.Start, nosepoke.End))
print("licks taken %d, licking time: %s" % (nosepoke.LickNumber, nosepoke.LickDuration))
print(sum(n.LickNumber for n in visit.Nosepokes) == visit.LickNumber)
print(sum((n.LickDuration for n in visit.Nosepokes), timedelta(0)) == visit.LickDuration)
print(sum((n.LickContactTime for n in visit.Nosepokes), timedelta(0)) == visit.LickContactTime)
print(sum((n.Duration for n in visit.Nosepokes), timedelta(0)) == visit.NosepokeDuration)
print(nosepoke.Visit is visit)

## Combining data from multiple sources
There is often many data files recorded from one experiment. To merge them into one object you have to load them first into a _`Loader`_ and then create an object of _`Merger`_ class.

To obtain list of files matching 'C57\_AB/*.zip' pattern you might use the [_`glob`_ module of The Python Standard Library](https://docs.python.org/2/library/glob.html).

In [ ]:
import glob
dataFiles = glob.glob('C57_AB/*.zip')

loaders = [pm.Loader(filename) for filename in dataFiles]
mm = pm.Merger(*loaders)
n = len(mm.getVisits())
print("Total number of visits in data: %d" % n)
print(n == sum(len(ml.getVisits()) for ml in loaders))